<a href="https://colab.research.google.com/github/vsrivatsan2096/iku/blob/master/Iku_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Libraries

In [1]:
!pip install -U -q numpy
!pip install -U -q keras
!pip install -U -q scikit-learn
!pip install -U -q matplotlib
!pip install -U -q nltk
!pip install -U -q PyDrive 
!pip install -U -q pandas
!pip install -U -q https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-win_amd64.whl
!pip install -U -q torchvision
!pip install --quiet tensorflow-hub
!pip install --quiet seaborn
!pip install --quiet "tensorflow>=1.7"

    100% |████████████████████████████████| 17.3MB 2.3MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
    100% |████████████████████████████████| 1.4MB 11.0MB/s 
    100% |████████████████████████████████| 993kB 19.8MB/s 
    100% |████████████████████████████████| 10.1MB 4.1MB/s 
torch-1.0.1-cp36-cp36m-win_amd64.whl is not a supported wheel on this platform.


# Google Drive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import json

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_ids = ["1JslQ15FgNAcEdCsKGAGkOjOBBTq_jaUw", "1q7pTs8JeD7o5v2P38C0FrbXJGErPFfWZ"]
file_names = ['google-embeddings-w2v.gz', 'common-crawl-embeddings-w2v.txt']

# Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Cloning Github

In [0]:
!git clone https://github.com/vsrivatsan2096/iku.git

# Pre-Trained embeddings

In [0]:
# Google
download = drive.CreateFile({'id':file_ids[0]})
download.GetContentFile(file_names[0])
google_embeddings = 'google-embeddings-w2v.gz'

In [0]:
# Common Crawl
download = drive.CreateFile({'id':file_ids[1]})
download.GetContentFile(file_names[1])
common_crawl_embeddings = 'common-crawl-embeddings-w2v.txt'

In [0]:
!wget https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
infersent_embeddings = 'infersent1.pkl'

# Dataset

In [0]:
file_ids = ["1bqdQccb6176JhXU7CZbWnv3vTAi7BH_-"]
file_names = ['questions.csv']

In [0]:
for each_id, each_name in zip(file_ids, file_names):
    download = drive.CreateFile({'id':each_id})
    download.GetContentFile(each_name)

In [0]:
del download
del gauth
del drive

In [0]:
import numpy as np
import pandas as pd
import multiprocessing

In [0]:
questions = pd.read_csv("questions.csv")
questions.dropna(inplace=True)

In [0]:
questions1 = questions.iloc[:, 3].values
questions2 = questions.iloc[:, 4].values
is_duplicate_questions = questions.iloc[:, 5].values

In [13]:
questions.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
cores = multiprocessing.cpu_count()

In [0]:
length = is_duplicate_questions.shape[0]

In [0]:
X_scores = {}

# Scores

In [0]:
# https://github.com/makcedward/nlp/blob/master/sample/nlp-word_mover_distance.ipynb

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import jaccard_similarity_score
from sklearn.neighbors import DistanceMetric

In [0]:
def cosine_similarity(self, sentence1, sentence2):
    return cosine_similarity(sentence1.reshape(1, -1), sentence2.reshape(1, -1))

In [0]:
def manhattan_distances(self, sentence1, sentence2):
    return manhattan_distances(sentence1.reshape(1, -1), sentence2.reshape(1, -1))

In [0]:
def euclidean_distances(self, sentence1, sentence2):
    return euclidean_distances(sentence1.reshape(1, -1), sentence2.reshape(1, -1))

In [0]:
def jaccard_similarity(self, sentence1, sentence2):
    jac_score = 0
    try:
        jac_score = jaccard_similarity_score(sentence1.reshape(1, -1), sentence2.reshape(1, -1))
    finally:
        return jac_score

In [0]:
def minkowski_distances(self, sentence1, sentence2):
    minkowski_distance = DistanceMetric.get_metric('minkowski')
    return minkowski_distance.pairwise(sentence1.reshape(1, -1), sentence2.reshape(1, -1))

In [0]:
def get_scores(self, sentences1, sentences2, length):
    scores = []
    for each in range(length):
        each_score = []
        each_score.append(self.cosine_similarity(sentences1[each], sentences2[each])[0][0])
        each_score.append(self.manhattan_distances(sentences1[each], sentences2[each])[0][0])
        each_score.append(self.euclidean_distances(ssentences1[each], sentences2[each])[0][0])
        #each_score.append(self.jaccard_similarity(self.sentences1[each], self.sentences2[each]))
        each_score.append(self.minkowski_distances(sentences1[each], sentences2[each])[0][0])
        scores[each] = np.asarray(each_score)
    return scores

# Natural Language Processing

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess

In [17]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
lemma = WordNetLemmatizer()
stopword = stopwords.words("english")

In [0]:
sentences_1 = []
for i in questions1:
    tempx = re.sub(r"[^A-Za-z]", " ", str(i))
    tempx = tempx.lower().split()
    tempx = [word for word in tempx if word not in stopword]
    tempx = [lemma.lemmatize(word, pos="a") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="r") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="n") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="v") for word in tempx]
    sentences_1.append(" ".join(tempx))
sentences_1 = np.asarray(sentences_1)

In [0]:
sentences_2 = []
for i in questions2:
    tempx = re.sub(r"[^A-Za-z]", " ", str(i))
    tempx = tempx.lower().split()
    tempx = [word for word in tempx if word not in stopword]
    tempx = [lemma.lemmatize(word, pos="a") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="r") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="n") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="v") for word in tempx]
    sentences_2.append(" ".join(tempx))
sentences_2 = np.asarray(sentences_2)

# LSA Method

http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True)

In [0]:
svd_model = TruncatedSVD(n_components=300,
                         algorithm='randomized',
                         n_iter=10, random_state=42)

In [0]:
lsa_model1 = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

In [0]:
lsa_model2 = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

In [0]:
lsa_1 = lsa_model1.fit_transform(sentences_1)
lsa_2 = lsa_model2.fit_transform(sentences_2)

In [0]:
X_scores['lsa'] = get_scores(lsa_1, lsa_2)

In [65]:
del vectorizer
del svd_model
del lsa_model1
del lsa_model2
del lsa_1
del lsa_2

NameError: ignored

# Word2Vec model(Using Mean to get the sentence vectors)

http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

In [0]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [0]:
google_model = KeyedVectors.load_word2vec_format("E:\Models\pre_trained\word2vec\google\google.300d.bin", binary=True)
#wiki_model = KeyedVectors.load_word2vec_format("models/pretrained/glove/wiki/wiki.300d.txt", binary=False)
#common_crawl_model = KeyedVectors.load_word2vec_format("models/pretrained/glove/common_crawl/common_crawl.300d.txt", binary=False)

In [0]:
def sentence_vectorizer(model, sentence):
    vectors =[]
    num = 0
    for i in sentence.split():
        try:
            if num == 0:
                vectors = model[i]
            else:
                vectors = np.add(vectors, model[i])
            num += 1
        except:
            pass
    return np.array(vectors) / num

In [0]:
sent_vec1 = []
for each in sentences_1:
    temp = sentence_vectorizer(google_model, each)
    if temp.shape[0] != 0:
        sent_vec1.append(temp)
    else:
        sent_vec1.append(np.zeros((300,)))
sent_vec1 = np.asarray(sent_vec1)

In [0]:
sent_vec2 = []
for each in sentences_2:
    temp = sentence_vectorizer(google_model, each)
    if temp.shape[0] != 0:
        sent_vec2.append(temp)
    else:
        sent_vec2.append(np.zeros((300,)))
sent_vec2 = np.asarray(sent_vec2)

In [0]:
X_scores['sentence vector'] = get_scores(sent_vec1, sent_vec2)

In [0]:
del sent_vec1
del sent_vec2

# InferText model

https://github.com/facebookresearch/InferSent

In [0]:
import torch
import numpy as np
from random import randint

**Load Model**

In [0]:
from models.infersent.models import InferSent
model_version = 1
MODEL_PATH = "models\infersent\infersent%s.pickle" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

In [0]:
use_cuda = False
model = model.cuda() if use_cuda else model # Keep it on CPU or put it on GPU

In [0]:
W2V_PATH = 'E:\Models\pre_trained\glove\commoncrawl\common_crawl.300d.txt'
model.set_w2v_path(W2V_PATH)

In [0]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


**Encode Sentences**

In [0]:
infersent_1 = model.encode(sentences_1[:20], bsize=128, tokenize=False, verbose=True)

Nb words kept : 139/152 (91.4%)
Speed : 25.0 sentences/s (cpu mode, bsize=128)


In [0]:
infersent_2 = model.encode(sentences_2[:20], bsize=128, tokenize=False, verbose=True)

Nb words kept : 140/150 (93.3%)
Speed : 35.7 sentences/s (cpu mode, bsize=128)


In [0]:
X_scores['sentence vector'] = get_scores(infersent_1, infersent_2)

In [0]:
del model
del infersent_1
del infersent_2

# Sentence Encoder V2

https://tfhub.dev/google/universal-sentence-encoder/2

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
embed = hub.Module(module_url)

In [0]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings_1 = session.run(embed(sentences_1))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0310 13:10:26.869572  7768 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [0]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings_2 = session.run(embed(sentences_2))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0310 13:10:56.016236  7768 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [0]:
X_scores['sentence encoder'] = get_scores(infersent_1, infersent_2)

In [0]:
del sentences_embeddings_1
del sentences_embeddings_2

# CNN and TimeDistributed

https://www.kaggle.com/zhihang/an-ensemble-approach-cnn-and-timedistributed

In [0]:
import numpy as np
import pandas as pd

In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [0]:
import re
import matplotlib.pyplot as plt
import datetime, time, json
from string import punctuation

In [0]:
from collections import defaultdict
from sklearn.metrics import accuracy_score

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

In [0]:
all_questions = sentences_1.tolist() + sentences_2.tolist()

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_questions)

In [0]:
question1_word_sequences = tokenizer.texts_to_sequences(sentences_1.tolist())
question2_word_sequences = tokenizer.texts_to_sequences(sentences_2.tolist())

In [0]:
word_index = tokenizer.word_index

In [0]:
max_question_len = 0
for each in range(length):
    max_question_len = max(max_question_len, len(question1_word_sequences[each]), len(question2_word_sequences[each]))
print(max_question_len)

97


In [0]:
train_q1 = pad_sequences(question1_word_sequences,
                              maxlen = max_question_len)

In [0]:
train_q2 = pad_sequences(question2_word_sequences,
                              maxlen = max_question_len)

In [0]:
embeddings_index = {}
with open(common_crawl_embeddings, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
print('Word embeddings:', len(embeddings_index))

Word embeddings: 2196016


In [0]:
embedding_dim = 300

In [0]:
nb_words = len(word_index)
word_embedding_matrix = np.zeros((nb_words + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        word_embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0)) #75,334

Null word embeddings: 16082


In [0]:
units = 128 # Number of nodes in the Dense layers
dropout = 0.25 # Percentage of nodes to drop
nb_filter = 32 # Number of filters to use in Convolution1D
filter_length = 3 # Length of filter for Convolution1D

In [0]:
weights = initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=2)
bias = bias_initializer='zeros'

In [0]:
from keras.layers import Input

In [0]:
model_1_input = Input(shape = (max_question_len,), dtype = 'int32', name = 'model_1_input')
model_1_embedding = Embedding(nb_words + 1,
                     embedding_dim,
                     weights = [word_embedding_matrix], 
                     input_length = max_question_len,
                     trainable = False)(model_1_input)
model_1_conv_a = Convolution1D(filters = nb_filter, 
                         kernel_size = filter_length, 
                         padding = 'same')(model_1_embedding)
model_1_batch_a = BatchNormalization()(model_1_conv_a)
model_1_act = Activation('relu')(model_1_batch_a)
model_1_drop_a = Dropout(dropout)(model_1_act)
model_1_conv_b = Convolution1D(filters = nb_filter, 
                         kernel_size = filter_length, 
                         padding = 'same')(model_1_drop_a)
model_1_batch_b = BatchNormalization()(model_1_conv_b)
model_1_act_b = Activation('relu')(model_1_batch_b)
model_1_drop_b = Dropout(dropout)(model_1_act_b)
model_1_flat = Flatten()(model_1_drop_b)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model_2_input = Input(shape = (max_question_len,), dtype = 'int32', name = 'model_2_input')
model_2_embedding = Embedding(nb_words + 1,
                     embedding_dim,
                     weights = [word_embedding_matrix], 
                     input_length = max_question_len,
                     trainable = False)(model_2_input)
model_2_conv_a = Convolution1D(filters = nb_filter, 
                         kernel_size = filter_length, 
                         padding = 'same')(model_2_embedding)
model_2_batch_a = BatchNormalization()(model_2_conv_a)
model_2_act = Activation('relu')(model_2_batch_a)
model_2_drop_a = Dropout(dropout)(model_2_act)
model_2_conv_b = Convolution1D(filters = nb_filter, 
                         kernel_size = filter_length, 
                         padding = 'same')(model_2_drop_a)
model_2_batch_b = BatchNormalization()(model_2_conv_b)
model_2_act_b = Activation('relu')(model_2_batch_b)
model_2_drop_b = Dropout(dropout)(model_2_act_b)
model_2_flat = Flatten()(model_2_drop_b)

In [0]:
model_3_input = Input(shape = (max_question_len,), dtype = 'int32', name = 'model_3_input')
model_3_embedding = Embedding(nb_words + 1,
                     embedding_dim,
                     weights = [word_embedding_matrix],
                     input_length = max_question_len,
                     trainable = False)(model_3_input)
model_3_time_distributed = TimeDistributed(Dense(embedding_dim))(model_3_embedding)
model_3_batch = BatchNormalization()(model_3_time_distributed)
model_3_act = Activation('relu')(model_3_batch)
model_3_drop = Dropout(dropout)(model_3_act)
model_3_lambda = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim, ))(model_3_drop)

In [0]:
model_4_input = Input(shape = (max_question_len,), dtype = 'int32', name = 'model_4_input')
model_4_embedding = Embedding(nb_words + 1,
                     embedding_dim,
                     weights = [word_embedding_matrix],
                     input_length = max_question_len,
                     trainable = False)(model_4_input)
model_4_time_distributed = TimeDistributed(Dense(embedding_dim))(model_4_embedding)
model_4_batch = BatchNormalization()(model_4_time_distributed)
model_4_act = Activation('relu')(model_4_batch)
model_4_drop = Dropout(dropout)(model_4_act)
model_4_lambda = Lambda(lambda x: K.max(x, axis=1), output_shape=(embedding_dim, ))(model_4_drop)

In [0]:
from keras.layers import concatenate

In [0]:
merge_layer = concatenate([model_1_flat, model_2_flat, model_3_lambda, model_4_lambda], name = 'merge_layer')

In [0]:
t = Dense(200, activation = 'relu', name = 'dense1')(merge_layer)
t = Dropout(0.3)(t)
t = BatchNormalization()(t)

In [0]:
t = Dense(200, activation = 'relu', name  ='dense2')(t)
t = Dropout(0.3)(t)
t = BatchNormalization()(t)

In [0]:
t = Dense(100, activation= 'relu',name = 'dense3')(t)
t = Dropout(0.3)(t)
t = BatchNormalization()(t)

In [0]:
final_output = Dense(1, activation = 'sigmoid')(t)

In [0]:
from keras.models import Model

In [0]:
model = Model(inputs = [model_1_input, model_2_input, model_3_input, model_4_input], outputs = final_output)

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
t0 = time.time()
history = model.fit([train_q1, train_q2, train_q1, train_q2],
                    questions.is_duplicate,
                    batch_size=256,
                    epochs=128, #Use 100, I reduce it for Kaggle,
                    verbose=True,
                    shuffle=True)
t1 = time.time()
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Epoch 1/128
404348/404348 [==============================] - 133s 328us/step - loss: 0.4278 - acc: 0.7913
Epoch 2/128
404348/404348 [==============================] - 130s 321us/step - loss: 0.4168 - acc: 0.7978
Epoch 3/128
404348/404348 [==============================] - 130s 322us/step - loss: 0.4066 - acc: 0.8036
Epoch 4/128
404348/404348 [==============================] - 130s 321us/step - loss: 0.3979 - acc: 0.8096
Epoch 5/128
404348/404348 [==============================] - 130s 322us/step - loss: 0.3910 - acc: 0.8136
Epoch 6/128
404348/404348 [==============================] - 130s 321us/step - loss: 0.3843 - acc: 0.8170
Epoch 7/128
404348/404348 [==============================] - 130s 322us/step - loss: 0.3773 - acc: 0.8212
Epoch 8/128
404348/404348 [==============================] - 130s 321us/step - loss: 0.3726 - acc: 0.8245
Epoch 9/128
404348/404348 [==============================] - 130s 321us/step - loss: 0.3681 - acc: 0.8269
Epoch 10/128
404348/404348 [==================

In [0]:
model.save_weights("cnn_td_quora_len_97.h5")

In [0]:
json = model.to_json()

In [0]:
with open("cnn_td_quora_len_97.json", "w") as json_file:
    json_file.write(json)

In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('cnn_td_quora_len_97.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1F4ONz6EEh-KojZTilrcjBsWXA89fvblq


In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('cnn_td_quora_len_97.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 166gELhGcUW69T3NPxouzjQtoh8-QTvtB


In [0]:
del embeddings_index
del embedding
del model
del train_q1
del train_q2
del word_embedding_matrix
del question1_word_sequences
del question2_word_sequences
del all_questions
del uploaded

# Siamese Neural Networks(Using LSTM and GRU)

https://medium.com/mlreview/implementing-malstm-on-kaggles-quora-question-pairs-competition-8b31b0b16a07

In [0]:
from keras.models import Model
import keras.backend as backend
from keras.layers import Input, Embedding, LSTM, Lambda, GRU, Dropout

In [0]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']

In [0]:
q2n_left = []
for sentence in sentences_1.tolist():
    temp_sentence = []
    for word in sentence.split():
        if word not in vocabulary:
            vocabulary[word] = len(inverse_vocabulary)
            temp_sentence.append(len(inverse_vocabulary))
            inverse_vocabulary.append(word)
        else:
            temp_sentence.append(vocabulary[word])
    q2n_left.append(temp_sentence)

In [0]:
q2n_right = []
for sentence in sentences_2.tolist():
    temp_sentence = []
    for word in sentence.split():
        if word not in vocabulary:
            vocabulary[word] = len(inverse_vocabulary)
            temp_sentence.append(len(inverse_vocabulary))
            inverse_vocabulary.append(word)
        else:
            temp_sentence.append(vocabulary[word])
    q2n_right.append(temp_sentence)

In [0]:
embedding_dim = 300
embeddings = np.zeros((len(vocabulary) + 1, embedding_dim))
embeddings[0] = 0

In [0]:
from gensim.models.keyedvectors import KeyedVectors

In [0]:
google_model = KeyedVectors.load_word2vec_format(google_embeddings, binary=True)

In [0]:
for word, index in vocabulary.items():
    if word in google_model.vocab:
        embeddings[index] = google_model.word_vec(word)

In [0]:
del google_model

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
dataset_left = q2n_left

In [0]:
dataset_right = q2n_right

In [0]:
max_seq_length = 0
for each in range(length):
    max_seq_length = max(max_seq_length, len(q2n_left[each]), len(q2n_right[each]))
print(max_seq_length)

97


In [0]:
dataset_left = pad_sequences(q2n_left, maxlen=max_seq_length)
dataset_right = pad_sequences(q2n_right, maxlen=max_seq_length)

In [0]:
dataset_left.shape == dataset_right.shape

True

In [0]:
n_hidden1 = 512
n_hidden2 = 384
n_hidden3 = 256
n_hidden4 = 128

In [0]:
left_input = Input(shape=(max_seq_length, ), dtype='int32')
right_input = Input(shape=(max_seq_length, ), dtype='int32')

In [0]:
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], 
                            input_length=max_seq_length, trainable=False)

In [0]:
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

In [0]:
shared_lstm1 = LSTM(n_hidden1, return_sequences=True)
shared_dropout1 = Dropout(0.3)
shared_gru1 = GRU(n_hidden2, return_sequences=True)
shared_dropout2 = Dropout(0.4)
shared_gru2 = GRU(n_hidden3, return_sequences=True)
shared_dropout3 = Dropout(0.3)
shared_lstm2 = LSTM(n_hidden4, return_sequences=False)

In [0]:
left_lstm1 = shared_lstm1(encoded_left)
left_dropout1 = shared_dropout1(left_lstm1)
left_gru1 = shared_gru1(left_dropout1)
left_dropout2 = shared_dropout2(left_gru1)
left_gru2 = shared_gru2(left_dropout2)
left_dropout3 = shared_dropout3(left_gru2)
left_lstm2 = shared_lstm2(left_dropout3)

In [0]:
right_lstm1 = shared_lstm1(encoded_right)
right_dropout1 = shared_dropout1(right_lstm1)
right_gru1 = shared_gru1(right_dropout1)
right_dropout2 = shared_dropout2(right_gru1)
right_gru2 = shared_gru2(right_dropout2)
right_dropout3 = shared_dropout3(right_gru2)
right_lstm2 = shared_lstm2(right_dropout3)

In [0]:
manhattan_distance_for_lstm = Lambda(function=lambda x: backend.exp(-backend.sum(backend.abs(x[0]-x[1]), axis=1, keepdims=True)),
                                     output_shape=lambda x: (x[0][0], 1))([left_lstm2, right_lstm2])

In [0]:
siamese_network = Model([left_input, right_input], manhattan_distance_for_lstm)
siamese_network.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])
siamese_network.fit([dataset_left, dataset_right], is_duplicate_questions, batch_size=256, 
                        epochs=1)

Epoch 1/1
404348/404348 [==============================] - 3060s 8ms/step - loss: 0.1753 - acc: 0.7357


In [0]:
siamese_network.save_weights("siamese_network_quora_len_97.h5")

In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('siamese_network_quora_len_97.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
json_file = siamese_network.to_json()

In [0]:
with open("siamese_network_quora_len_97.json", "w") as json:
    json_file.write(json)

In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('siamese_network_quora_len_97.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
del vocabulary
del inverse_vocabulary
del q2n_left
del q2n_right
del embedding_dim
del embeddings
del dataset_left
del dataset_right
del siamese_network

# Siamese Network of CNN, Time Distributed, RNN and Attention Model

In [0]:
import pandas as pd
import numpy as np

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, Lambda, TimeDistributed, Convolution1D, Flatten, Activation, BatchNormalization, Dropout
from keras import backend as K
from keras.models import Model

In [0]:
all_questions = sentences_1.tolist() + sentences_2.tolist()

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_questions)

In [0]:
question1_word_sequences = tokenizer.texts_to_sequences(sentences_1.tolist())
question2_word_sequences = tokenizer.texts_to_sequences(sentences_2.tolist())

In [0]:
word_index = tokenizer.word_index

In [34]:
max_question_len = 0
for each in range(length):
    max_question_len = max(max_question_len, len(question1_word_sequences[each]), len(question2_word_sequences[each]))
print(max_question_len)

97


In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
train_q1 = pad_sequences(question1_word_sequences,
                              maxlen = max_question_len)

In [0]:
train_q2 = pad_sequences(question2_word_sequences,
                              maxlen = max_question_len)

In [38]:
embeddings_index = {}
with open('common-crawl-embeddings-w2v.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
print('Word embeddings:', len(embeddings_index))

Word embeddings: 2196016


In [0]:
embedding_dim = 300

In [0]:
nb_words = len(word_index)
word_embedding_matrix = np.zeros((nb_words + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        word_embedding_matrix[i] = embedding_vector

In [41]:
print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0)) #75,334

Null word embeddings: 16082


In [0]:
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    
    def dot_product(self, x, kernel):
        if K.backend() == 'tensorflow':
            return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
        else:
            return K.dot(x, kernel)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = self.dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = self.dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
left_input = Input(shape=(max_question_len, ), dtype='int32')
right_input = Input(shape=(max_question_len, ), dtype='int32')

In [0]:
shared_embedding_layer = Embedding(nb_words + 1, embedding_dim,
                                   weights = [word_embedding_matrix], 
                                   input_length = max_question_len,
                                   trainable = False)

In [0]:
units = 128 # Number of nodes in the Dense layers
dropout = 0.25 # Percentage of nodes to drop
nb_filter = 128 # Number of filters to use in Convolution1D
filter_length = 3 # Length of filter for Convolution1D

In [0]:
shared_conv_layer = Convolution1D(filters = nb_filter, 
                            kernel_size = filter_length,
                            padding = 'same')

In [0]:
shared_dropout_1 = Dropout(dropout)

In [0]:
shared_batch_normalization_layer = BatchNormalization()

In [0]:
shared_activation_layer = TimeDistributed(Dense(128, activation='relu'))

In [0]:
shared_time_distributed_dense_layer = TimeDistributed(Flatten())

In [0]:
shared_bidirectional_lstm_layer = Bidirectional(LSTM(units, return_sequences=True))

In [0]:
shared_dropout_2 = Dropout(dropout)

In [0]:
shared_bidirectional_gru_layer = Bidirectional(GRU(units, return_sequences=True))

In [0]:
shared_dropout_3 = Dropout(dropout)

In [0]:
shared_attention_layer = AttentionWithContext()

**Siamese Network**

In [56]:
embedded_left = shared_embedding_layer(left_input)
embedded_right = shared_embedding_layer(right_input)

Instructions for updating:
Colocations handled automatically by placer.


In [57]:
cnn_left = shared_conv_layer(embedded_left)
dropout_1_left = shared_dropout_1(cnn_left)
batch_norm_left = shared_batch_normalization_layer(dropout_1_left)
activation_left = shared_activation_layer(batch_norm_left)
time_distributed_dense_left = shared_time_distributed_dense_layer(activation_left)
bidirectional_lstm_left = shared_bidirectional_lstm_layer(time_distributed_dense_left)
dropout_2_left = shared_dropout_2(bidirectional_lstm_left)
bidirectional_gru_left = shared_bidirectional_gru_layer(dropout_2_left)
dropout_3_left = shared_dropout_3(bidirectional_gru_left)
attention_left = shared_attention_layer(dropout_3_left)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
cnn_right = shared_conv_layer(embedded_right)
dropout_1_right = shared_dropout_1(cnn_right)
batch_norm_right = shared_batch_normalization_layer(cnn_right)
activation_right = shared_activation_layer(batch_norm_right)
time_distributed_dense_right = shared_time_distributed_dense_layer(activation_right)
bidirectional_lstm_right = shared_bidirectional_lstm_layer(time_distributed_dense_right)
dropout_2_right = shared_dropout_2(bidirectional_lstm_right)
bidirectional_gru_right = shared_bidirectional_gru_layer(dropout_2_right)
dropout_3_right = shared_dropout_3(bidirectional_gru_right)
attention_right = shared_attention_layer(dropout_3_right)

In [0]:
output_layer = Lambda(function=lambda x: K.exp(-K.sum(K.abs(x[0]-x[1]), axis=1, keepdims=True)),
                                     output_shape=lambda x: (x[0][0], 1))([attention_left, attention_right])

In [0]:
model = Model([left_input, right_input], output_layer)

In [61]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 97)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 97, 300)      19608000    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 97)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 97, 128)      115328      embedding_1[0][0]                
          

In [0]:
model.compile(loss='mean_squared_error', optimizer="adam", metrics=['accuracy'])

In [63]:
model.fit([train_q1, train_q2], is_duplicate_questions, batch_size=256, 
                        epochs=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
404348/404348 [==============================] - 2525s 6ms/step - loss: 0.1705 - acc: 0.7443


In [0]:
model.save_weights('super_siamese_quora_len_97.h5')

In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('super_siamese_quora_len_97.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
json_file = siamese_network.to_json()

In [0]:
with open("super_siamese_quora_len_97.json", "w") as json:
    json_file.write(json)

In [0]:
uploaded = drive.CreateFile()
uploaded.SetContentFile('super_siamese_quora_len_97.json')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
del embeddings_index
del embedding
del model
del train_q1
del train_q2
del word_embedding_matrix
del question1_word_sequences
del question2_word_sequences
del all_questions

# Ensembling

**Dataset Creation**

In [0]:
X_scores = pd.DataFrame(X_scores)

In [0]:
X_scores.head()

**XG Boosting**

https://www.programcreek.com/python/example/99824/xgboost.XGBClassifier

In [0]:
import xgboost as xgb

In [0]:
xgb_classifier = xgb.XGBClassifier(max_depth=7, min_child_weight=1, learning_rate=0.1,
                           n_estimators=500, silent=True, objective='binary:logistic',
                           gamma=0, max_delta_step=0, subsample=1, colsample_bytree=1,
                           colsample_bylevel=1, reg_alpha=0, reg_lambda=0,
                           scale_pos_weight=1, seed=1, missing=None)

In [0]:
xgb_classifier.fit(X_scores, is_duplicate_questions, verbose=True)